In [2]:
import subprocess
from glob import glob
import os
from multiprocessing import Process, Pipe
import subprocess
import multiprocessing
import CarbonaraDataTools as cdt
import numpy as np

### **Fix the PDB**
If the PDB contains chain with missing residues or atoms, using OpenMM's PDBFixer to repair and infer full structures. Just remember this isnt a miracle performer and usually only suitable for small segments of missing structure!

In [20]:
# try:
#     from pdbfixer import PDBFixer
# except ImportError:
#     !conda install -c conda-forge pdbfixer

# try:
#     from openmm.app import PDBFile
# except ImportError:
#     !conda install -c omnia openmm


# fixer = PDBFixer(filename='user_preprocessed_data/3v03_bsa.pdb')

# fixer.findMissingResidues()
# fixer.findMissingAtoms()
# fixer.addMissingAtoms()
# PDBFile.writeFile(fixer.topology, fixer.positions, open('user_preprocessed_data/3v03_bsa_fixed.pdb', 'w'))

### **Setup Directories**
This section sets up the master directory (contains all refinements) and the refinement directory (this is the name of the specific structure we want to refine!). 


In [ ]:
fit_master_name = 'newFitData' # default name
fit_master_dir = cdt.setup_fit_master_dir(root_dir = os.getcwd(), fit_master_name = fit_master_name)

refinement_directory_name = 'bsa_validation' # change to name of your molecule
refine_dir = cdt.setup_refinement_dir(refinement_directory_name, fit_master_dir)


### **Load User Data**
This section loads the user's pre-processed data, which includes pdb files and a scatter profile file.


In [ ]:
my_data_folder = 'user_preprocessed_data/' # containing pdb, saxs profile, (optionally) secondary structure, (optionally) varying sections
user_raw_data_folder = os.path.join( os.getcwd(),my_data_folder)

pdb_files = glob(user_raw_data_folder+'*.pdb') # currently selects ALL .pdb files in your data folder (is this what you want?)
saxs_file = my_data_folder + '3v03_bsa_foxs_saxs.dat' # specific saxs file name


### **Extract Structure from PDB**
For a single PDB (for now), for each chain in the PDB: pull out the xyz positions of alpha carbons, the sequence, infer the secondary structure (currently geometrically - DSSP or self defined possible) and any missing residues (maybe go back and fix your structure!)

In [ ]:
coords_chains, sequence_chains, secondary_structure_chains, missing_residues_chains = cdt.pull_structure_from_pdb(pdb_files[0]) 


### **Check for Missing Residues**
This section checks for segments in the chain(s) geometrically, if a distance greater than 10 Å between sequential alpha carbons is present


In [ ]:
for coords in coords_chains:
    breaking_indices = cdt.missing_ca_check(coords, threshold_dist_Å=10)
    if len(breaking_indices) > 0:
        print("Missing segments of chain found: ", len(breaking_indices), breaking_indices)

### **Write Fingerprint File**
Write the sequence and secondary structure for each chain to the `fingerPrint1.dat` file (in the refinement directory). This is the format expected by our fitting algorithm!

In [ ]:
number_of_chains = len(coords_chains)
fingerprint_file = cdt.write_fingerprint_file(number_chains = number_of_chains, sequence = sequence_chains,
                           secondary_structure = secondary_structure_chains, working_path = refine_dir)

### **Sanity Check**
Lets perform a sanity check, the lengths should all be the same for: coords, sequence, secondary structure

In [ ]:
for c, s, ss in zip(coords_chains, sequence_chains, secondary_structure_chains):
    print("coords: ", len(c), "-*- sequence: ", len(s), "-*- secondary structure: ", len(ss))

### **Write Coordinates to Refinement Directory**
Write each chain to its own xyz file in the refinement directory, for 3 chains expect `coordinates1.dat`, `coordinates2.dat` &  `coordinates3.dat`

In [ ]:
coords_files = []
for i, coords in enumerate( coords_chains ):
    coords_files.append( cdt.write_coordinates_file(coords, working_path=refine_dir, carb_index=i+1) )

### **Write Mixture File to Refinement Directory**
Auto-Mixture file isnt fully implemented

In [ ]:
mixture_file = cdt.write_mixture_file(working_path=refine_dir)

### **Write SAXS File to Refinement Directory**
Writes the SAXS file to the required format refinement directory as `Saxs.dat`.

In [ ]:
scatter_file = cdt.write_saxs(saxs_file, working_path=refine_dir)

In [24]:

# >> Better distance constraints setup process - currently horrible
# contactPreds = [[231,686],[228,683],[225,591],[136,680],[830,890],[375,435],[149,205],[604,660],[719,784],[477,551],[22,96],[264,329]]
# fixedDistSet=[]
#if I wanted to fix these distances as 6 a fixedDistSet = [6.0,6.0,6.0,6.0]
# cdt.translate_distance_constraints(contactPreds,coords,splitList,working_path,fixedDistSet)



coords:  583 -*- sequence:  583 -*- secondary structure:  583


### **Find and write varying sections**
Automatically find varying secondary sections

In [25]:
varying_linker_chains = []
for coord_file in coords_files: 
    varying_linker_chains.append( cdt.auto_select_varying_linker(coord_file, fingerprint_file) )

varying_section_files = []
for varying_linkers in varying_linker_chains:
    varying_section_files.append( cdt.write_varysections_file(varying_linkers, refine_dir) )

100%|██████████| 36/36 [00:01<00:00, 28.79it/s]


In [26]:
def visualise_varying_sections(coords_chains, secondary_structure_chains, varying_linker_chains, chain_index=1):

    x_lst, y_lst, z_lst, color_lst =  cdt.smooth_me_varying(coords_chains[chain_index], secondary_structure_chains[chain_index], varying_linker_chains[chain_index], oversample=5)
    structure_fig = cdt.line_plotly(x_lst, y_lst, z_lst, color_lst, outline=True)
    structure_fig.update_layout(height=600)
    structure_fig.show()

In [27]:
visualise_varying_sections(coords_chains, secondary_structure_chains, varying_linker_chains, 0)

In [28]:
# need to now figure out how to set off carbonara from python
runs_dirs = setup_runs(refine_dir, number_runs = 3)

In [29]:
runs_dirs

['/Users/josh/Documents/PhD/DevDungeon/carbonara/newFitData/bsa_validation/run1',
 '/Users/josh/Documents/PhD/DevDungeon/carbonara/newFitData/bsa_validation/run2',
 '/Users/josh/Documents/PhD/DevDungeon/carbonara/newFitData/bsa_validation/run3']

In [30]:
def indv_carbonara_cmd(
                        scatter_file, 
                        file_locs, 
                        resume_tag, 
                        paired_predictions, 
                        varying_sections, 
                        no_structures,
                        within_monomer_hydro_cover, 
                        between_monomer_hydro_cover, 
                        kmin, 
                        kmax, 
                        max_no_fit_steps, 
                        prediction_file,
                        scatter_out, 
                        mixture_file, 
                        prev_fit_str, 
                        log_loc, 
                        end_line_prev_log, 
                        affine_trans, 
                        number_runs=3, 
                        verbose=False,
                        carbonara_executable="./predictStructureQvary"
                    ):
    """
    Constructs a command to execute the Carbonara program with the provided arguments.

    Parameters:
    scatter_file (str): The file path containing scatter data.
    file_locs (str): Refinement directory file path.
    resume_tag (str): A tag to resume a previous operation.
    paired_predictions (str): Add intra-chain distance constraints when predicting multiple chain structures. 
    varying_sections (str): The index of secondary sections of the protein structure that should be resampled in fitting.
    no_structures (int): Number of chain structures in multimer (monomer = 1) (dimer = 2) (trimer = 3) ( ... ).
    within_monomer_hydro_cover (str): hydrophobic cover within a monomer - not used.
    between_monomer_hydro_cover (str): hydrophobic cover between monomers - not used.
    kmin (float): Minimum angle of scattering (k or q) used to fit.
    kmax (float): Maximum angle of scattering (k or q) used to fit.
    max_no_fit_steps (int): Maximum number of steps before finishing fit attempt.
    prediction_file (str): File path (with mol prefix) to store the xyz predictions (/path/to/prediction/molX).
    scatter_out (str): Output file for simulated scatter fit (/path/to/simulated/scatter/scatterX).
    mixture_file (str): File path to a mixture of multi- monomers/dimers/... 
    prev_fit_str (str): Previous fit string (not really used)
    log_loc (str): Location of the log output file (/path/to/log/fitLogX).
    end_line_prev_log (str): End line of the previous log (not really used)
    affine_trans (str): Affine transformation (true or false) - used only for multimers.
    number_runs (int, optional): Number of sequential runs. Defaults to 1.
    verbose (bool, optional): Flag for verbose output. Defaults to False.
    carbonara_executable (str, optional): Path to the carbonara executable. Defaults to "./predictStructureQvary".

    Returns:
    list: A list containing the command to be executed and its arguments.
    """

    # Constructing the command with carbonara executable and arguments
    cmd = [
        carbonara_executable,
        scatter_file,
        file_locs,
        resume_tag,
        paired_predictions,
        varying_sections,
        str(no_structures),
        within_monomer_hydro_cover,
        between_monomer_hydro_cover,
        str(kmin),
        str(kmax),
        str(max_no_fit_steps),
        prediction_file,
        scatter_out,
        mixture_file,
        prev_fit_str,
        log_loc,
        end_line_prev_log,
        str(affine_trans)
    ]

    return cmd


def run_command(cmd, conn):
    result = subprocess.run(cmd, capture_output=True, text=True)
    conn.send({'cmd': cmd, 'stdout': result.stdout, 'stderr': result.stderr, 'returncode': result.returncode})
    conn.close()

def track_commands(commands):
    processes = []
    parent_connections = []

    # Start all processes
    for cmd in commands:
        parent_conn, child_conn = Pipe()
        parent_connections.append(parent_conn)

        process = Process(target=run_command, args=(cmd, child_conn,))
        processes.append(process)
        process.start()

    # Wait for all processes to finish and get their results
    results = [parent_conn.recv() for parent_conn in parent_connections]

    for process in processes:
        process.join()

    return results

In [31]:
### argv[ 3] restart tag (use to start from existing prediction)
resume_tag = "frompdb"

### argv[ 4] paired distances file (can be empty)
pair_contact_restraint_bool = "False"

### argv[ 5] varying sections file (again can be empty)
varying_file = varying_section_files[0]

### argv[ 7] request to apply hydrophobic covering WITHIN monomers will be a list of sections on which to apply it -- Currently not used
withinMonomerHydroCover="none"

### argv[ 8] request to apply hydrophobic covering BETWEEN monomers will be a list of pairs to try to hydropobically pair -- currently not used
betweenMonomerHydroCover="none"

### argv[ 9] kmin
kmin=0.01

### argv[10] kmax
kmax=0.25

### argv[11] Max number of fitting steps
maxNoFitSteps=500

seq_run_index = 1
carb_index = 0
prediction_file_out = os.path.join( runs_dirs[carb_index], "mol" + str(seq_run_index) )
scatter_file_out = os.path.join( runs_dirs[carb_index], "scatter" + str(seq_run_index) )
log_file_out = os.path.join( runs_dirs[carb_index], "fitLog" + str(seq_run_index) )

run_cmd = indv_carbonara_cmd(scatter_file = scatter_file, resume_tag = resume_tag, file_locs = refine_dir+'/', paired_predictions = pair_contact_restraint_bool, 
                             varying_sections = varying_file, no_structures = number_of_chains, 
                             within_monomer_hydro_cover = withinMonomerHydroCover, between_monomer_hydro_cover = betweenMonomerHydroCover,
                             kmin = kmin, kmax = kmax, max_no_fit_steps = maxNoFitSteps, prediction_file = prediction_file_out,
                             scatter_out = scatter_file_out, mixture_file = mixture_file, prev_fit_str = "not_needed", log_loc = log_file_out,
                             end_line_prev_log = "null", affine_trans = "False", number_runs=1, verbose=False)



In [32]:
run_cmd

['./predictStructureQvary',
 '/Users/josh/Documents/PhD/DevDungeon/carbonara/newFitData/bsa_validation/Saxs.dat',
 '/Users/josh/Documents/PhD/DevDungeon/carbonara/newFitData/bsa_validation/',
 'frompdb',
 'False',
 '/Users/josh/Documents/PhD/DevDungeon/carbonara/newFitData/bsa_validation/varyingSectionSecondary1.dat',
 '1',
 'none',
 'none',
 '0.01',
 '0.25',
 '500',
 '/Users/josh/Documents/PhD/DevDungeon/carbonara/newFitData/bsa_validation/run1/mol1',
 '/Users/josh/Documents/PhD/DevDungeon/carbonara/newFitData/bsa_validation/run1/scatter1',
 '/Users/josh/Documents/PhD/DevDungeon/carbonara/newFitData/bsa_validation/mixtureFile.dat',
 'not_needed',
 '/Users/josh/Documents/PhD/DevDungeon/carbonara/newFitData/bsa_validation/run1/fitLog1',
 'null',
 'False']

In [33]:
def write_runme(working_path, fit_name, fit_n_times, min_q, max_q, max_fit_steps, pairedQ=False, rotation=False):

    curr = os.getcwd()
    script_name = '/RunMe_bsa.sh'
    run_file = curr + script_name

    with open(run_file, 'w+') as fout:

        saxs_file = 'Saxs.dat'
        FP_file = "fingerPrint1.dat"
        coords_file = 'coordinates1.dat'
        varying_file = "varyingSectionSecondary1.dat"
        mixture_file = "mixtureFile.dat"

        # $1 is the molecule name $2 the output file $3 the restart file if true

        fout.write('#!/bin/bash')
        fout.write('\n ROOT = '+working_path)
        fout.write('\n')

        fout.write('\n ### argv[ 1] scattering data file')
        fout.write('\n ScatterFile='+working_path+saxs_file)
        fout.write('\n')

        fout.write('\n ### argv[ 2] sequence file location')
        fout.write('\n fileLocs='+working_path)
        fout.write('\n')

        fout.write('\n ### argv[ 3] restart tag (use to start from existing prediction)')
        fout.write('\n initialCoordsFile=frompdb')
        fout.write('\n')
        
        fout.write('\n ### argv[ 4] paired distances file (can be empty)')
        if pairedQ==False:
            fout.write('\n pairedPredictions=False')
        else:
            fout.write('\n pairedPredictions=True')
        fout.write('\n')
        
        fout.write('\n ### argv[ 5] fixed sections file (again can be empty)')
        fout.write('\n fixedsections='+working_path+varying_file)
        fout.write('\n')

        fout.write('\n ### argv[ 6] number of structures')
        fout.write('\n noStructures=1')
        fout.write('\n')

        fout.write('\n ### argv[ 7] request to apply hydrophobic covering WITHIN monomers will be a list of sections on which to apply it -- Currently not used')
        fout.write('\n withinMonomerHydroCover=none')
        fout.write('\n')

        fout.write('\n ### argv[ 8] request to apply hydrophobic covering BETWEEN monomers will be a list of pairs to try to hydropobically pair -- currently not used')
        fout.write('\n betweenMonomerHydroCover=none')
        fout.write('\n')

        fout.write('\n ### argv[ 9] kmin')
        fout.write('\n kmin='+str(min_q))
        fout.write('\n')

        fout.write('\n ### argv[10] kmax')
        fout.write('\n kmax='+str(max_q))
        fout.write('\n')

        fout.write('\n ### argv[11] Max number of fitting steps')
        fout.write('\n maxNoFitSteps='+str(max_fit_steps))
        fout.write('\n')

        fout.write('\n ### argv[12] prediction file - mol[i] in the fitting folder')
        fout.write('\n predictionFile='+working_path+fit_name)
        fout.write('\n')

        fout.write('\n ### argv[13] scattering output file')
        fout.write('\n scatterOut='+working_path+fit_name)
        fout.write('\n')

        fout.write('\n ### argv[14] mixture list file, alist of sets of numbers indicatig the allowed set of mixture percentages of each species (e.g. dimer 20 monomer 80)')
        fout.write('\n mixtureFile='+working_path+mixture_file)
        fout.write('\n')

        fout.write('\n ### argv[15] previous fit string in form fitname/mol6Substep_10_1.dat+fitname/mol6Substep_10_2.dat')
        fout.write('\n prevFitStr='+working_path+'redundant')
        fout.write('\n')

        fout.write('\n ### argv[16] log file location')
        fout.write('\n logLoc='+working_path+fit_name)
        fout.write('\n')

        fout.write('\n ### argv[17] last line of the previous fit log, this is only used for a restart if argv[3] = True')
        fout.write('\n endLinePrevLog=null')
        fout.write('\n')
                   
        fout.write('\n ### argv[18] is true if we want to apply affine rotations,false if not.')
        if rotation==False:
            fout.write('\n affineTrans=False')
        else:
            fout.write('\n affineTrans=True')
        fout.write('\n')


        fout.write('\nfor i in {1..'+str(fit_n_times)+'}')

        fout.write('\n\ndo')
        fout.write('\n\n   echo " Run number : $i "')
        fout.write('\n\n   ./predictStructureQvary $ScatterFile $fileLocs $initialCoordsFile $pairedPredictions $fixedsections $noStructures $withinMonomerHydroCover $betweenMonomerHydroCover $kmin $kmax $maxNoFitSteps $predictionFile/mol$i $scatterOut/scatter$i.dat $mixtureFile $prevFitStr $logLoc/fitLog$i.dat $endLinePrevLog $affineTrans')

        fout.write('\n\ndone')

    # return script_name[1:]
    # print('Successfully written bash script to: ', run_file)
        fout.close()
        # now write the run folder if its empty

        working = working_path+fit_name
        if os.path.exists(working):
            skip = True
        else:
            os.makedirs(working)

    # os.mkdir(working_path+'/fitdata')

In [34]:
# write_runme(refine_dir+'/', 'bsa_validation', 2, 0.01, 0.25, 50, pairedQ=False, rotation=False)

In [35]:
dists = np.linalg.norm( np.diff(coords_chains[0],axis=0), axis=1) 

In [36]:
dists[dists>4]

array([], dtype=float64)

In [4]:
import smtplib
from email.mime.text import MIMEText

subject = "Carbonara Job Update"
body = "Your carbonara is complete! Please check your fitting file: /where/it/at "
sender = "carbonara.job.notification@gmail.com"
recipients = ["josh.j.mckeown@durham.ac.uk", "christopher.prior@durham.ac.uk", "arron.n.bale@durham.ac.uk"]
password = "zmod rgsw zkqn xugt"


def send_email(subject, body, sender, recipients, password):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(recipients)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
       smtp_server.login(sender, password)
       smtp_server.sendmail(sender, recipients, msg.as_string())
    print("Message sent!")


send_email(subject, body, sender, recipients, password)


Message sent!


In [7]:
import math 
import numpy as np
def dotproduct(v1, v2):
  return sum((a*b) for a, b in zip(v1, v2))


def length(v):
  return math.sqrt(dotproduct(v, v))

In [13]:
v1 = [1,2,3]
v2 = [1,2,3]

dotproduct(v1, v2)


14

In [14]:
np.dot(v1,v2)

14

In [15]:
length(v1)

3.7416573867739413

In [16]:
np.linalg.norm(v1)

3.7416573867739413